In [50]:
import wandb
import pandas as pd
import numpy as np
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Whisker
from bokeh.transform import factor_cmap, jitter
from bokeh.layouts import gridplot

In [64]:
api = wandb.Api()
runs = api.runs("hmr24/FundusDRGrading-src_fundusClassif_scripts")

important_keys = ['lr', 'ema', 'swa', 'mixup', 'preprocessing', 'as_regression']
list_configs = []
for i, r in enumerate(runs):
    kappa = r.summary_metrics.get("Validation Quadratic Kappa", None)
    if kappa is None:
        continue
    config = {k: v for k, v in r.config.items() if k in important_keys}
    config['name'] = r.name
    config['kappa'] = kappa
    list_configs.append(config)
    
    if i >= 50:
        break
df = pd.DataFrame(list_configs)

df['ema'] = df['ema'].map({1: 'EMA', 0: 'No EMA'})
df['swa'] = df['swa'].map({1: 'SWA', 0: 'No SWA'})
df['mixup'] = df['mixup'].map({1: 'Mixup', 0: 'No Mixup'})
df['as_regression'] = df['as_regression'].map({1: 'Regression', 0: 'Classification'})
df

,lr,ema,swa,mixup,as_regression,preprocessing,name,kappa
0,0.000028,EMA,No SWA,No Mixup,Classification,clahe_rgb,misty-sweep-51,0.882492
1,0.000038,No EMA,No SWA,No Mixup,Regression,sarki,light-sweep-50,0.714634
2,0.000048,EMA,No SWA,No Mixup,Classification,clahe_rgb,firm-sweep-49,0.958879
3,0.000029,No EMA,No SWA,No Mixup,Classification,absent,usual-sweep-48,0.919654
4,0.000036,EMA,No SWA,No Mixup,Regression,absent,devoted-sweep-47,0.673747
5,0.000035,No EMA,No SWA,No Mixup,Regression,clahe_lab,light-sweep-46,0.904181
6,0.000030,EMA,SWA,No Mixup,Regression,sarki,valiant-sweep-45,0.674597
7,0.000029,No EMA,No SWA,Mixup,Classification,sarki,faithful-sweep-44,0.708022
8,0.000031,No EMA,No SWA,No Mixup,Regression,clahe_rgb,iconic-sweep-43,0.897591
9,0.000029,EMA,SWA,Mixup,Classification,sarki,blooming-sweep-42,0.567482


In [81]:
def get_graph(df, key):
    g = df.groupby(key)

    upper = g.kappa.quantile(0.80)
    lower = g.kappa.quantile(0.20)


    datasource = ColumnDataSource(df)
    output_notebook()

    classes = list(df[key].unique())
    p = figure(height=500, x_range=classes, title=f"Kappa by {key.upper()}", tools='')

    source = ColumnDataSource(data=dict(base=classes, upper=upper, lower=lower))

    error = Whisker(base="base", upper="upper", lower="lower", source=source,
                    level="annotation", line_width=2)

    error.upper_head.size=20
    error.lower_head.size=20

    p.add_layout(error)

    p.scatter(jitter(key, 0.1, range=p.x_range), y='kappa', 
            source=datasource, 
            size=10, 
            line_color="white",
            color=factor_cmap(key, "Light7", classes),
            alpha=0.6)
    return p


p_ema = get_graph(df, 'ema')
p_swa = get_graph(df, 'swa')
p_mixup = get_graph(df, 'mixup')
p_as_regression = get_graph(df, 'as_regression')

grid = gridplot([[p_ema, p_swa], [p_mixup, p_as_regression]])
show(grid)

In [79]:
r.config

{'lr': 3.407674268842685e-05,
 'ema': 0,
 'swa': 1,
 'mixup': 1,
 'define': [512, 512],
 'optimizer': 'AdamW',
 'mixup_alpha': 0.2750413850688639,
 'training/lr': 3.407674268842685e-05,
 'cutmix_alpha': 0.4213267053431159,
 'datasets/ddr': '/home/florian/Documents/donnees_medicales/DDR-dataset/DR_grading/',
 'as_regression': 0,
 'data/img_size': [512, 512],
 'preprocessing': 'autobalance',
 'data/drop_last': True,
 'data/use_cache': True,
 'datasets/aptos': '/home/florian/Documents/donnees_medicales/aptos/',
 'datasets/idrid': '/home/florian/Documents/donnees_medicales/IDRID/B. Disease Grading/',
 'logger/project': 'Grading-DiabeticRetinopathy-Comparisons-V3',
 'model/img_size': [512, 512],
 'data/batch_size': 48,
 'data/valid_size': 0.2,
 'trainer/devices': [0],
 'data/num_workers': 8,
 'datasets/eyepacs': '/home/florian/Documents/donnees_medicales/eyepacs/',
 'model/pretrained': True,
 'data/cache_option': 'disk',
 'model/num_classes': 5,
 'trainer/precision': '16-mixed',
 'model/arc